In [ ]:
!pip install mlflow

In [26]:
!pip install tabulate


[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [53]:
import mlflow
import pandas as pd
from tabulate import tabulate
import json
import os

# Set the tracking URI to the MLFlow Server
mlflow.set_tracking_uri('http://127.0.0.1:5000')

# Retrieve experiments
experiments = mlflow.search_experiments()

# Initialize a list to collect all run data
all_runs = []

# Retrieve runs from each experiment and store them
for experiment in experiments:
    experiment_id = experiment.experiment_id

    if experiment.name == "base_4o" or experiment.name == "DUP":
        runs = mlflow.search_runs(experiment_ids=[experiment_id])

        for _, run in runs.iterrows():
            if 'params.accuracy' in run:
                try:
                    accuracy = float(run['params.accuracy'])
                except ValueError:
                    accuracy = None
                run_data = {
                    'experiment_id': experiment_id,
                    'experiment_name': experiment.name,
                    'run_id': run.run_id,
                    'run_name': run['tags.mlflow.runName'],
                    'accuracy': accuracy
                }
                all_runs.append(run_data)
            else:
                print(f"No accuracy metric found for run {run.run_id} in experiment: {experiment.name}")

# Convert runs to a DataFrame for easier manipulation
runs_df = pd.DataFrame(all_runs)

# Function to extract and standardize the subject name
def extract_subject_name(run_name):
    if 'base_4o' in run_name:
        return run_name.split('base_4o_')[1].rsplit('_', 1)[0].replace('_test', '')
    elif 'majority_vote' in run_name:
        return run_name.split('majority_vote_')[1].rsplit('_', 1)[0]
    elif '4o_' in run_name:
        return run_name.split('4o_')[1].rsplit('_', 1)[0].replace('_test.csv', '')
    return run_name

# Apply the function to standardize subject names
runs_df['subject'] = runs_df['run_name'].apply(extract_subject_name)

# Ensure the accuracy column is numeric
runs_df['accuracy'] = pd.to_numeric(runs_df['accuracy'], errors='coerce')

# Filter for relevant experiments
base_4o_df = runs_df[runs_df['experiment_name'] == 'base_4o'][['subject', 'accuracy', 'run_id', 'experiment_id']]
dup_majority_vote_df = runs_df[(runs_df['experiment_name'] == 'DUP') & runs_df['run_name'].str.startswith('majority_vote_')][['subject', 'accuracy', 'run_id', 'experiment_id']]
dup_4o_df = runs_df[(runs_df['experiment_name'] == 'DUP') & runs_df['run_name'].str.startswith('4o_')][['subject', 'accuracy', 'run_id', 'experiment_id']]

# Rename accuracy columns for merging
base_4o_df.rename(columns={'accuracy': 'base_4o', 'run_id': 'base_4o_run_id', 'experiment_id': 'base_4o_experiment_id'}, inplace=True)
dup_majority_vote_df.rename(columns={'accuracy': '1_layer_dup+majority_vote', 'run_id': '1_layer_dup+majority_vote_run_id', 'experiment_id': '1_layer_dup+majority_vote_experiment_id'}, inplace=True)
dup_4o_df.rename(columns={'accuracy': '1_layer_DUP', 'run_id': '1_layer_DUP_run_id', 'experiment_id': '1_layer_DUP_experiment_id'}, inplace=True)

# Perform the merging step-by-step
merged_df = pd.merge(base_4o_df, dup_majority_vote_df, on='subject', how='outer')
merged_df = pd.merge(merged_df, dup_4o_df, on='subject', how='outer')

# Replace NaN values with '-' for better readability
merged_df.fillna('-', inplace=True)

# Ensure the subjects are sorted alphabetically for consistency
merged_df.sort_values(by='subject', inplace=True)

# TextGrad results
textgrad_results = {
    "abstract_algebra": 0.81,
    "anatomy": None,
    "astronomy": None,
    "business_ethics": None,
    "clinical_knowledge": None,
    "college_biology": None,
    "college_chemistry": 0.69,
    "college_computer_science": 0.89,
    "college_mathematics": 0.87,
    "college_medicine": None,
    "college_physics": None,
    "computer_security": 0.89,
    "conceptual_physics": None,
    "econometrics": 0.75438596,
    "electrical_engineering": 0.82758621,
    "elementary_mathematics": 0.97619048,
    "formal_logic": 0.86507937,
    "global_facts": 0.68,
    "high_school_biology": None,
    "high_school_chemistry": None,
    "high_school_computer_science": None,
    "high_school_european_history": None,
    "high_school_geography": None,
    "high_school_government_and_politics": None,
    "high_school_macroeconomics": None,
    "high_school_mathematics": 0.8962963,
    "high_school_microeconomics": None,
    "high_school_physics": None,
    "high_school_psychology": None,
    "high_school_statistics": 0.91203704,
    "high_school_us_history": None,
    "high_school_world_history": None,
    "human_aging": None,
    "human_sexuality": None,
    "international_law": None,
    "jurisprudence": None,
    "logical_fallacies": None,
    "machine_learning": 0.86607143,
    "management": None,
    "marketing": None,
    "medical_genetics": None,
    "miscellaneous": None,
    "moral_disputes": None,
    "moral_scenarios": None,
    "nutrition": None,
    "philosophy": None,
    "prehistory": None,
    "professional_accounting": None,
    "professional_law": None,
    "professional_medicine": None,
    "professional_psychology": None,
    "public_relations": None,
    "security_studies": 0.79183673,
    "sociology": None,
    "us_foreign_policy": None,
    "virology": 0.55421687,
    "world_religions": None
}

# Convert the TextGrad results to a DataFrame
textgrad_df = pd.DataFrame(list(textgrad_results.items()), columns=['subject', 'TextGrad Vanilla Instance Optimization'])

# Merge the TextGrad results with the merged_df DataFrame
merged_df = pd.merge(merged_df, textgrad_df, on='subject', how='left')

# Replace NaN values in TextGrad with '-' for better readability
merged_df['TextGrad Vanilla Instance Optimization'].fillna('-', inplace=True)

# Adjust pandas display settings to show all rows
pd.set_option('display.max_rows', None)

# Display the merged DataFrame using tabulate
print("\nMerged DataFrame:")
print(tabulate(merged_df, headers='keys', tablefmt='psql'))



Merged DataFrame:
+----+-------------------------------------+-----------+----------------------------------+-------------------------+-----------------------------+------------------------------------+-------------------------------------------+---------------+----------------------------------+-----------------------------+------------------------------------------+
|    | subject                             |   base_4o | base_4o_run_id                   |   base_4o_experiment_id | 1_layer_dup+majority_vote   | 1_layer_dup+majority_vote_run_id   | 1_layer_dup+majority_vote_experiment_id   |   1_layer_DUP | 1_layer_DUP_run_id               |   1_layer_DUP_experiment_id | TextGrad Vanilla Instance Optimization   |
|----+-------------------------------------+-----------+----------------------------------+-------------------------+-----------------------------+------------------------------------+-------------------------------------------+---------------+-------------------------------

/var/folders/nz/6nwg7z1x7c9gx9p6ql0f018w0000gn/T/ipykernel_908/669393339.py:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df.fillna('-', inplace=True)
/var/folders/nz/6nwg7z1x7c9gx9p6ql0f018w0000gn/T/ipykernel_908/669393339.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['TextGrad Vanilla Instance Optimization'].fillna('-', in

In [50]:
# Function to find the best performance and corresponding condition
def find_best_performance(row):
    conditions = ['base_4o', '1_layer_dup+majority_vote', '1_layer_DUP','TextGrad Vanilla Instance Optimization']
    best_value = -float('inf')
    best_condition = None
    for condition in conditions:
        value = row[condition]
        if value != '-' and value > best_value:
            best_value = value
            best_condition = condition
    return pd.Series({'best_performance': best_value, 'best_condition': best_condition})

# Apply the function to each row
best_df = df.apply(find_best_performance, axis=1)

# Add the subject column to the best_df
best_df['subject'] = df['subject']

# Reorder the columns for better readability
best_df = best_df[['subject', 'best_performance', 'best_condition']]

# Display the result using tabulate
print("\nBest Performance DataFrame:")
print(tabulate(best_df, headers='keys', tablefmt='psql'))


Best Performance DataFrame:
+----+-------------------------------------+--------------------+----------------------------------------+
|    | subject                             |   best_performance | best_condition                         |
|----+-------------------------------------+--------------------+----------------------------------------|
|  0 | abstract_algebra                    |           0.81     | TextGrad Vanilla Instance Optimization |
|  1 | anatomy                             |           0.896296 | base_4o                                |
|  2 | astronomy                           |           0.927632 | base_4o                                |
|  3 | business_ethics                     |           0.82     | 1_layer_DUP                            |
|  4 | clinical_knowledge                  |           0.909434 | base_4o                                |
|  5 | college_biology                     |           0.958333 | base_4o                                |
|  6 | c

In [69]:
# Apply the function to each row
best_df = merged_df.apply(find_best_performance, axis=1)

# Add the subject column to the best_df
best_df['subject'] = merged_df['subject']

# Add run_id and experiment_id for best condition
def get_best_condition_run_info(row):
    condition = row['best_condition']
    if condition == 'base_4o':
        run_id = merged_df.loc[merged_df['subject'] == row['subject'], 'base_4o_run_id'].values[0]
        experiment_id = merged_df.loc[merged_df['subject'] == row['subject'], 'base_4o_experiment_id'].values[0]
    elif condition == '1_layer_dup+majority_vote':
        run_id = merged_df.loc[merged_df['subject'] == row['subject'], '1_layer_dup+majority_vote_run_id'].values[0]
        experiment_id = merged_df.loc[merged_df['subject'] == row['subject'], '1_layer_dup+majority_vote_experiment_id'].values[0]
    elif condition == '1_layer_DUP':
        run_id = merged_df.loc[merged_df['subject'] == row['subject'], '1_layer_DUP_run_id'].values[0]
        experiment_id = merged_df.loc[merged_df['subject'] == row['subject'], '1_layer_DUP_experiment_id'].values[0]
    elif condition == 'TextGrad Vanilla Instance Optimization':
        return pd.Series({'run_id': 'TextGrad Vanilla Instance Optimization', 'experiment_id': 'TextGrad Vanilla Instance Optimization'})
    return pd.Series({'run_id': run_id, 'experiment_id': experiment_id})

best_df[['run_id', 'experiment_id']] = best_df.apply(get_best_condition_run_info, axis=1)

# Ensure the artifact paths are correct and handle non-existent artifacts properly
def get_questions_and_results(run_id, experiment_id, subject):
    artifact_path = f'{experiment_id}/{run_id}/artifacts/results/{subject}_test_answers.json'
    print(f"Attempting to fetch artifact at: {artifact_path}")
    try:
        artifact_uri = mlflow.artifacts.download_artifacts(run_id=run_id, artifact_path=f"results/{subject}_test_answers.json")
        print(f"Successfully fetched artifact at: {artifact_path}")
        with open(artifact_uri, 'r') as file:
            data = json.load(file)
            # Check if the 'data' key exists and contains the required information
            if 'data' in data and isinstance(data['data'], list):
                questions = data['data']
                wrong_questions = [q for q in questions if not q[4]]  # The 'correct' field is the 5th column
                return wrong_questions
            else:
                print(f"Unexpected data format in artifact: {artifact_path}")
                return []
    except Exception as e:
        print(f"Artifact not found: {artifact_path}. Error: {str(e)}")
        return []

# Add wrong_questions column
best_df['wrong_questions'] = best_df.apply(
    lambda row: get_questions_and_results(row['run_id'], row['experiment_id'], row['subject']) if row['best_condition'] != 'TextGrad Vanilla Instance Optimization' else "Manual handling required for TextGrad", 
    axis=1
)

# Display the final DataFrame using tabulate
print("\nBest Performance DataFrame with Wrong Questions:")
print(tabulate(best_df, headers='keys', tablefmt='psql'))

# Save the DataFrame to a CSV file to review
best_df.to_csv('/mnt/data/best_performance_with_wrong_questions.csv', index=False)

Attempting to fetch artifact at: 925317739931269583/90949b2acb314ccfb972640228993e8e/artifacts/results/anatomy_test_answers.json
Successfully fetched artifact at: 925317739931269583/90949b2acb314ccfb972640228993e8e/artifacts/results/anatomy_test_answers.json
Attempting to fetch artifact at: 925317739931269583/b1f743f36b1f46e9a1b6abd9131ad1ce/artifacts/results/astronomy_test_answers.json
Successfully fetched artifact at: 925317739931269583/b1f743f36b1f46e9a1b6abd9131ad1ce/artifacts/results/astronomy_test_answers.json
Attempting to fetch artifact at: 472027273416396635/df3f4b67adeb466586087bc75eaaf9a3/artifacts/results/business_ethics_test_answers.json
Artifact not found: 472027273416396635/df3f4b67adeb466586087bc75eaaf9a3/artifacts/results/business_ethics_test_answers.json. Error: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/472027273416396635/df3f4b67adeb466586087bc75eaaf9a3/artifacts:
##### File r

OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [35]:
# Calculate the average of the best performance scores
average_best_performance = best_df['best_performance'].mean()

# Print the result
print(f"\nAverage of the best performance scores: {average_best_performance:.6f}")

# Calculate the average of the best performance scores
average_4o = merged_df['base_4o'].mean()

# Print the result
print(f"\nAverage of the 4o scores: {average_4o:.6f}")


Average of the best performance scores: 0.885416

Average of the 4o scores: 0.865110
